# CUDA 数组接口

由于将数据从CPU移动到GPU的开销很大，我们希望尽可能多地保持数据始终位于GPU上。

有时在我们的工作流程中，我们也想改变使用的工具。也许我们使用`cupy`加载了一个数据数组，但我们想用`numba`编写一个自定义的CUDA内核。或者我们想切换到使用像`pytorch`这样的深度学习框架。

当这些库中的任何一个将数据加载到GPU上时，内存中的数组基本上是相同的。cupy的`ndarray`和numba的`DeviceNDArray`之间的区别仅在于数组如何被包装并连接到Python。

幸运的是，借助像[DLPack](https://github.com/dmlc/dlpack)和[`__cuda_array__interface__`](https://numba.readthedocs.io/en/stable/cuda/cuda_array_interface.html)这样的实用工具，我们可以在不修改GPU上数据的情况下从一种类型转换为另一种类型。我们只需创建一个新的Python包装器对象并传输所有设备指针。

确保流行的GPU Python库之间的兼容性是RAPIDS社区的核心目标之一。

![](images/array-interface.png)

让我们看看这个功能的实际应用！

我们首先使用CuPy创建一个数组。

In [7]:
import cupy as cp
# 使用更合理的数组大小以避免内存问题
cp_arr = cp.random.random((100, 1000, 1000))
cp_arr

array([[[0.17572606, 0.38152158, 0.64269351, ..., 0.07515534,
         0.99049298, 0.94435744],
        [0.26558308, 0.79105898, 0.5082067 , ..., 0.88454661,
         0.28137788, 0.37358692],
        [0.38617845, 0.76164433, 0.17188144, ..., 0.34715918,
         0.38098977, 0.30668737],
        ...,
        [0.60317778, 0.69273163, 0.39666527, ..., 0.51775052,
         0.37331455, 0.59810914],
        [0.4196741 , 0.9361488 , 0.21997834, ..., 0.29165336,
         0.24868757, 0.8217988 ],
        [0.15465472, 0.37924016, 0.50648938, ..., 0.76492635,
         0.22148569, 0.47322672]],

       [[0.43089097, 0.87470549, 0.07045724, ..., 0.1061153 ,
         0.75396014, 0.40997852],
        [0.91328209, 0.18884004, 0.83711646, ..., 0.09092838,
         0.14908714, 0.94945127],
        [0.64622264, 0.75736539, 0.69856194, ..., 0.01780518,
         0.49111054, 0.61353111],
        ...,
        [0.69348589, 0.63368185, 0.57559398, ..., 0.79274069,
         0.85933448, 0.81995581],
        [0.5

In [8]:
type(cp_arr)

cupy.ndarray

现在让我们将其转换为Numba数组。

In [9]:
from numba import cuda
from numba import config as numba_config
numba_config.CUDA_ENABLE_PYNVJITLINK = True

numba_arr = cuda.to_device(cp_arr)
numba_arr

_请注意GPU内存使用量保持不变。这是因为`cp_arr`和`numba_arr`都引用相同的基础数据数组，但属于不同的类型。_

我们也可以将数组转换为PyTorch的`Tensor`对象。

In [10]:
import torch  # Requires pytorch

# 在Jupyter Notebook中，需要先初始化PyTorch的CUDA上下文
# 这样可以避免后续使用CUDA数组接口时出现初始化错误
torch.cuda.init()
print(f'PyTorch CUDA已初始化')
print(f'CUDA设备: {torch.cuda.get_device_name(0)}')

PyTorch CUDA已初始化
CUDA设备: Tesla T4


In [11]:
# 方法1：直接转换Numba数组到PyTorch（推荐）
torch_arr = torch.as_tensor(numba_arr, device='cuda')
torch_arr

# 方法2：如果直接转换失败，可以通过CuPy中转
# cp_from_numba = cp.asarray(numba_arr)
# torch_arr = torch.as_tensor(cp_from_numba, device='cuda')

tensor([[[0.1757, 0.3815, 0.6427,  ..., 0.0752, 0.9905, 0.9444],
         [0.2656, 0.7911, 0.5082,  ..., 0.8845, 0.2814, 0.3736],
         [0.3862, 0.7616, 0.1719,  ..., 0.3472, 0.3810, 0.3067],
         ...,
         [0.6032, 0.6927, 0.3967,  ..., 0.5178, 0.3733, 0.5981],
         [0.4197, 0.9361, 0.2200,  ..., 0.2917, 0.2487, 0.8218],
         [0.1547, 0.3792, 0.5065,  ..., 0.7649, 0.2215, 0.4732]],

        [[0.4309, 0.8747, 0.0705,  ..., 0.1061, 0.7540, 0.4100],
         [0.9133, 0.1888, 0.8371,  ..., 0.0909, 0.1491, 0.9495],
         [0.6462, 0.7574, 0.6986,  ..., 0.0178, 0.4911, 0.6135],
         ...,
         [0.6935, 0.6337, 0.5756,  ..., 0.7927, 0.8593, 0.8200],
         [0.5022, 0.1590, 0.5830,  ..., 0.8100, 0.7517, 0.9852],
         [0.1132, 0.5081, 0.5408,  ..., 0.5652, 0.3255, 0.9539]],

        [[0.2953, 0.8398, 0.6406,  ..., 0.5766, 0.9725, 0.2591],
         [0.7241, 0.1605, 0.8779,  ..., 0.1691, 0.1138, 0.9335],
         [0.5914, 0.6377, 0.3122,  ..., 0.0512, 0.9340, 0.

In [12]:
type(torch_arr)

torch.Tensor